In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import os
import copy

In [5]:
# Define position-specific feature weights
POSITION_FEATURE_WEIGHTS = {
    'OFF': {
        'finishing': 4,
        'creativity': 2,
        'distribution': 1,
        'defense': 1,
        'duels': 2
    },
    'MID': {
        'finishing': 2,
        'creativity': 3,
        'distribution': 3,
        'defense': 2,
        'duels': 2
    },
    'DEF': {
        'finishing': 1,
        'creativity': 1,
        'distribution': 2,
        'defense': 4,
        'duels': 3
    }
}


In [6]:
def apply_position_weights(df, sheet_name, feature_cols):
    """
    Apply position-specific weights to features based on sheet name.
    
    Args:
        df: DataFrame to modify
        sheet_name: Name of the sheet (used to determine position)
        feature_cols: List of feature columns to weight
    
    Returns:
        Tuple of (weighted DataFrame, list of weighted feature columns)
    """
    # Create a copy to avoid modifying the original dataframe
    df_weighted = df.copy()
    weighted_feature_cols = []
    
    # Determine position from sheet name
    position = None
    if 'off' in sheet_name.lower():
        position = 'OFF'
    elif 'mid' in sheet_name.lower():
        position = 'MID'
    elif 'def' in sheet_name.lower():
        position = 'DEF'
    else:
        print(f"Warning: Could not determine position from sheet name '{sheet_name}'.")
        print("Using equal weights for all features.")
        return df_weighted, feature_cols  # Return original columns if position unknown
    
    print(f"Applying {position} weights to sheet '{sheet_name}'")
    weights = POSITION_FEATURE_WEIGHTS[position]
    
    # Apply weights to each feature column
    for feature in feature_cols:
        if feature in weights and feature in df_weighted.columns:
            feature_weight = weights[feature]
            df_weighted[f'{feature}_weighted'] = df_weighted[feature] * feature_weight
            weighted_feature_cols.append(f'{feature}_weighted')
            print(f"  - Applied weight {feature_weight} to '{feature}'")
    
    # Calculate weighted score
    if weighted_feature_cols:
        df_weighted['weighted_score'] = df_weighted[weighted_feature_cols].sum(axis=1)
        
        # Normalize the weighted score to be between 0 and 1
        min_score = df_weighted['weighted_score'].min()
        max_score = df_weighted['weighted_score'].max()
        if max_score > min_score:
            df_weighted['weighted_score'] = (df_weighted['weighted_score'] - min_score) / (max_score - min_score)
        
        print(f"  - Created 'weighted_score' column")
    
    return df_weighted, weighted_feature_cols

In [7]:
def train_and_evaluate_sheet(df, sheet_name, feature_cols, target_fifa, target_real, use_weighted_features=True):
    """
    Train and evaluate models on a single sheet's data.
    
    Args:
        df: DataFrame containing the sheet data
        sheet_name: Name of the sheet being processed
        feature_cols: List of feature columns to use
        target_fifa: Column name for FIFA rating target
        target_real: Column name for real-life rating target
        use_weighted_features: Whether to apply position-based weights
        
    Returns:
        Dictionary containing models and evaluation metrics
    """
    print(f"\n===== Processing Sheet: {sheet_name} =====")
    
    # Check if required columns exist
    missing_features = [col for col in feature_cols if col not in df.columns]
    if missing_features:
        print(f"Warning: Missing features in {sheet_name}: {missing_features}")
        available_features = [col for col in feature_cols if col in df.columns]
        if not available_features:
            print(f"Error: No valid features available in {sheet_name}. Skipping.")
            return None
        print(f"Proceeding with available features: {available_features}")
        feature_cols = available_features
    
    if target_fifa not in df.columns:
        print(f"Error: Target column '{target_fifa}' not found in {sheet_name}. Skipping FIFA model.")
        has_fifa_target = False
    else:
        has_fifa_target = True
        
    if target_real not in df.columns:
        print(f"Error: Target column '{target_real}' not found in {sheet_name}. Skipping Real model.")
        has_real_target = False
    else:
        has_real_target = True
        
    if not has_fifa_target and not has_real_target:
        print(f"Error: No target columns found in {sheet_name}. Skipping sheet.")
        return None
    
    # Apply position-specific weights if requested
    if use_weighted_features:
        df_weighted, weighted_feature_cols = apply_position_weights(df, sheet_name, feature_cols)
        
        # ignore for now 
        ## Add the weighted score as an additional feature
        #if 'weighted_score' in df_weighted.columns:
            #weighted_feature_cols.append('weighted_score')
            
        # Use weighted features if available, otherwise use original features
        if weighted_feature_cols:
            X = df_weighted[weighted_feature_cols]
            print(f"Using weighted features: {weighted_feature_cols}")
        else:
            X = df[feature_cols]
            print(f"Using original features: {feature_cols}")
            
        # For the targets, still use the original dataframe
        features_used = weighted_feature_cols
    else:
        # Use original features without weighting
        X = df[feature_cols]
        features_used = feature_cols
    
    results = {
        'sheet_name': sheet_name,
        'feature_cols': features_used,
    }
    
    # Initialize models
    rf_model = RandomForestRegressor(random_state=42)
    gb_model = GradientBoostingRegressor(random_state=42)  # Second model for comparison
    
    # Process FIFA target if available
    if has_fifa_target:
        y_fifa = df[target_fifa]
        
        # Split data
        X_train, X_test, y_fifa_train, y_fifa_test = train_test_split(
            X, y_fifa, test_size=0.2, random_state=42
        )
        
        # Train Random Forest model
        rf_model.fit(X_train, y_fifa_train)
        rf_preds = rf_model.predict(X_test)
        
        # Train Gradient Boosting model
        gb_model.fit(X_train, y_fifa_train)
        gb_preds = gb_model.predict(X_test)
        
        # Calculate metrics
        rf_fifa_r2 = r2_score(y_fifa_test, rf_preds)
        rf_fifa_rmse = np.sqrt(mean_squared_error(y_fifa_test, rf_preds))
        
        gb_fifa_r2 = r2_score(y_fifa_test, gb_preds)
        gb_fifa_rmse = np.sqrt(mean_squared_error(y_fifa_test, gb_preds))
        
        # Print results
        print(f"--- {sheet_name} - FIFA Rating Prediction ---")
        print(f"Random Forest R²: {rf_fifa_r2:.3f}, RMSE: {rf_fifa_rmse:.3f}")
        print(f"Gradient Boosting R²: {gb_fifa_r2:.3f}, RMSE: {gb_fifa_rmse:.3f}")
        
        # Plot feature importance for both models
        fig, axes = plt.subplots(1, 2, figsize=(18, 6))
        
        # Random Forest importance
        plot_importance(rf_model, features_used, f"Random Forest - FIFA Rating ({sheet_name})", ax=axes[0])
        
        # Gradient Boosting importance
        plot_importance(gb_model, features_used, f"Gradient Boosting - FIFA Rating ({sheet_name})", ax=axes[1])
        
        plt.tight_layout()
        plt.savefig(f"FIFA_importance_comparison_{sheet_name.replace(' ', '_')}.png")
        plt.close()
        
        # Store results
        results['rf_fifa_model'] = rf_model
        results['rf_fifa_r2'] = rf_fifa_r2
        results['rf_fifa_rmse'] = rf_fifa_rmse
        results['rf_fifa_importances'] = rf_model.feature_importances_
        
        results['gb_fifa_model'] = gb_model
        results['gb_fifa_r2'] = gb_fifa_r2
        results['gb_fifa_rmse'] = gb_fifa_rmse
        results['gb_fifa_importances'] = gb_model.feature_importances_
    
    # Process Real-Life target if available
    if has_real_target:
        y_real = df[target_real]
        
        # Split data
        X_train, X_test, y_real_train, y_real_test = train_test_split(
            X, y_real, test_size=0.2, random_state=42
        )
        
        # Train Random Forest model
        rf_model.fit(X_train, y_real_train)
        rf_preds = rf_model.predict(X_test)
        
        # Train Gradient Boosting model
        gb_model.fit(X_train, y_real_train)
        gb_preds = gb_model.predict(X_test)
        
        # Calculate metrics
        rf_real_r2 = r2_score(y_real_test, rf_preds)
        rf_real_rmse = np.sqrt(mean_squared_error(y_real_test, rf_preds))
        
        gb_real_r2 = r2_score(y_real_test, gb_preds)
        gb_real_rmse = np.sqrt(mean_squared_error(y_real_test, gb_preds))
        
        # Print results
        print(f"--- {sheet_name} - Real-Life Rating Prediction ---")
        print(f"Random Forest R²: {rf_real_r2:.3f}, RMSE: {rf_real_rmse:.3f}")
        print(f"Gradient Boosting R²: {gb_real_r2:.3f}, RMSE: {gb_real_rmse:.3f}")
        
        # Plot feature importance for both models
        fig, axes = plt.subplots(1, 2, figsize=(18, 6))
        
        # Random Forest importance
        plot_importance(rf_model, features_used, f"Random Forest - Real-Life Rating ({sheet_name})", ax=axes[0])
        
        # Gradient Boosting importance
        plot_importance(gb_model, features_used, f"Gradient Boosting - Real-Life Rating ({sheet_name})", ax=axes[1])
        
        plt.tight_layout()
        plt.savefig(f"Real_importance_comparison_{sheet_name.replace(' ', '_')}.png")
        plt.close()
        
        # Store results
        results['rf_real_model'] = rf_model
        results['rf_real_r2'] = rf_real_r2
        results['rf_real_rmse'] = rf_real_rmse
        results['rf_real_importances'] = rf_model.feature_importances_
        
        results['gb_real_model'] = gb_model
        results['gb_real_r2'] = gb_real_r2
        results['gb_real_rmse'] = gb_real_rmse
        results['gb_real_importances'] = gb_model.feature_importances_
    
    return results


In [8]:
def plot_combined_r2_rmse(all_results):
    """Plot combined R² and RMSE comparison for all models across all sheets"""
    if not all_results:
        print("No data for combined plots")
        return

    sheets = [r['sheet_name'] for r in all_results]
    x = np.arange(len(sheets))
    width = 0.2

    # Extract metrics (default to 0 if missing)
    rf_fifa_r2 = [r.get('rf_fifa_r2', 0) for r in all_results]
    gb_fifa_r2 = [r.get('gb_fifa_r2', 0) for r in all_results]
    rf_real_r2 = [r.get('rf_real_r2', 0) for r in all_results]
    gb_real_r2 = [r.get('gb_real_r2', 0) for r in all_results]

    rf_fifa_rmse = [r.get('rf_fifa_rmse', 0) for r in all_results]
    gb_fifa_rmse = [r.get('gb_fifa_rmse', 0) for r in all_results]
    rf_real_rmse = [r.get('rf_real_rmse', 0) for r in all_results]
    gb_real_rmse = [r.get('gb_real_rmse', 0) for r in all_results]

    fig, axs = plt.subplots(2, 1, figsize=(14, 12), sharex=True)

    # --- R² subplot ---
    axs[0].bar(x - 1.5 * width, rf_fifa_r2, width, label='RF FIFA')
    axs[0].bar(x - 0.5 * width, gb_fifa_r2, width, label='GB FIFA')
    axs[0].bar(x + 0.5 * width, rf_real_r2, width, label='RF Real')
    axs[0].bar(x + 1.5 * width, gb_real_r2, width, label='GB Real')
    axs[0].set_title("R² Scores by Sheet and Model")
    axs[0].set_ylabel("R²")
    axs[0].set_ylim(0, 1)
    axs[0].legend()
    axs[0].grid(axis='y', linestyle='--', alpha=0.6)

    # --- RMSE subplot ---
    axs[1].bar(x - 1.5 * width, rf_fifa_rmse, width, label='RF FIFA')
    axs[1].bar(x - 0.5 * width, gb_fifa_rmse, width, label='GB FIFA')
    axs[1].bar(x + 0.5 * width, rf_real_rmse, width, label='RF Real')
    axs[1].bar(x + 1.5 * width, gb_real_rmse, width, label='GB Real')
    axs[1].set_title("RMSE Scores by Sheet and Model")
    axs[1].set_ylabel("RMSE")
    axs[1].legend()
    axs[1].set_xticks(x)
    axs[1].set_xticklabels(sheets, rotation=45, ha="right")
    axs[1].grid(axis='y', linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.savefig("Combined_R2_RMSE_Comparison.png")
    plt.close()

In [9]:
def plot_importance(model, feature_names, title, ax=None):
    """Plot feature importance for a model"""
    importances = model.feature_importances_
    indices = np.argsort(importances)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.barh(range(len(indices)), importances[indices], align='center')
    ax.set_yticks(range(len(indices)))
    ax.set_yticklabels([feature_names[i] for i in indices])
    ax.set_title(title)
    ax.set_xlabel("Importance")
    
    return ax

def compare_sheets_results(all_results):
    """Create comparison plots and tables for all sheets"""
    if not all_results:
        print("No valid results to compare")
        return
    
    # Compare FIFA R² across sheets for both models
    sheets = [r['sheet_name'] for r in all_results if 'rf_fifa_r2' in r]
    if sheets:
        rf_fifa_r2_values = [r['rf_fifa_r2'] for r in all_results if 'rf_fifa_r2' in r]
        gb_fifa_r2_values = [r['gb_fifa_r2'] for r in all_results if 'gb_fifa_r2' in r]
        
        fig, ax = plt.subplots(figsize=(12, 7))
        x = np.arange(len(sheets))
        width = 0.35
        
        ax.bar(x - width/2, rf_fifa_r2_values, width, label='Random Forest')
        ax.bar(x + width/2, gb_fifa_r2_values, width, label='Gradient Boosting')
        
        ax.set_title("FIFA Rating Model - R² by Sheet and Algorithm")
        ax.set_ylabel("R²")
        ax.set_ylim(0, 1)  # R² is typically between 0 and 1
        ax.set_xticks(x)
        ax.set_xticklabels(sheets, rotation=45)
        ax.legend()
        
        plt.tight_layout()
        plt.savefig("FIFA_R2_model_comparison.png")
        plt.close()
    
    # Compare Real-Life R² across sheets for both models
    sheets = [r['sheet_name'] for r in all_results if 'rf_real_r2' in r]
    if sheets:
        rf_real_r2_values = [r['rf_real_r2'] for r in all_results if 'rf_real_r2' in r]
        gb_real_r2_values = [r['gb_real_r2'] for r in all_results if 'gb_real_r2' in r]
        
        fig, ax = plt.subplots(figsize=(12, 7))
        x = np.arange(len(sheets))
        width = 0.35
        
        ax.bar(x - width/2, rf_real_r2_values, width, label='Random Forest')
        ax.bar(x + width/2, gb_real_r2_values, width, label='Gradient Boosting')
        
        ax.set_title("Real-Life Rating Model - R² by Sheet and Algorithm")
        ax.set_ylabel("R²")
        ax.set_ylim(0, 1)  # R² is typically between 0 and 1
        ax.set_xticks(x)
        ax.set_xticklabels(sheets, rotation=45)
        ax.legend()
        
        plt.tight_layout()
        plt.savefig("Real_R2_model_comparison.png")
        plt.close()
    
    # Create detailed summary table
    summary_data = []
    for result in all_results:
        sheet_name = result['sheet_name']
        row = {'Sheet': sheet_name}
        
        # Add Random Forest results
        if 'rf_fifa_r2' in result:
            row['RF FIFA R²'] = f"{result['rf_fifa_r2']:.3f}"
            row['RF FIFA RMSE'] = f"{result['rf_fifa_rmse']:.3f}"
        else:
            row['RF FIFA R²'] = "N/A"
            row['RF FIFA RMSE'] = "N/A"
            
        if 'rf_real_r2' in result:
            row['RF Real R²'] = f"{result['rf_real_r2']:.3f}"
            row['RF Real RMSE'] = f"{result['rf_real_rmse']:.3f}"
        else:
            row['RF Real R²'] = "N/A"
            row['RF Real RMSE'] = "N/A"
        
        # Add Gradient Boosting results
        if 'gb_fifa_r2' in result:
            row['GB FIFA R²'] = f"{result['gb_fifa_r2']:.3f}"
            row['GB FIFA RMSE'] = f"{result['gb_fifa_rmse']:.3f}"
        else:
            row['GB FIFA R²'] = "N/A"
            row['GB FIFA RMSE'] = "N/A"
            
        if 'gb_real_r2' in result:
            row['GB Real R²'] = f"{result['gb_real_r2']:.3f}"
            row['GB Real RMSE'] = f"{result['gb_real_rmse']:.3f}"
        else:
            row['GB Real R²'] = "N/A"
            row['GB Real RMSE'] = "N/A"
            
        summary_data.append(row)
    
    # Create the summary DataFrame
    summary_df = pd.DataFrame(summary_data)
    
    # Add a row for the algorithm comparison (which performed better on average)
    if len(summary_data) > 0:
        # Calculate average performance across sheets
        rf_fifa_r2_avg = np.mean([r['rf_fifa_r2'] for r in all_results if 'rf_fifa_r2' in r])
        gb_fifa_r2_avg = np.mean([r['gb_fifa_r2'] for r in all_results if 'gb_fifa_r2' in r])
        
        rf_real_r2_avg = np.mean([r['rf_real_r2'] for r in all_results if 'rf_real_r2' in r])
        gb_real_r2_avg = np.mean([r['gb_real_r2'] for r in all_results if 'gb_real_r2' in r])
        
        # Create average row
        avg_row = {
            'Sheet': 'AVERAGE',
            'RF FIFA R²': f"{rf_fifa_r2_avg:.3f}",
            'GB FIFA R²': f"{gb_fifa_r2_avg:.3f}",
            'RF Real R²': f"{rf_real_r2_avg:.3f}",
            'GB Real R²': f"{gb_real_r2_avg:.3f}"
        }
        
        # Highlight which algorithm performed better on average
        fifa_winner = "Random Forest" if rf_fifa_r2_avg > gb_fifa_r2_avg else "Gradient Boosting"
        real_winner = "Random Forest" if rf_real_r2_avg > gb_real_r2_avg else "Gradient Boosting"
        
        # Add winning algorithm 
        avg_row['FIFA Winner'] = fifa_winner
        avg_row['Real Winner'] = real_winner
        
        # Add the average row to the DataFrame
        summary_df = pd.concat([summary_df, pd.DataFrame([avg_row])], ignore_index=True)
    
    print("\n===== Summary of Results =====")
    print(summary_df)
    
    # Save summary to CSV
    summary_df.to_csv("model_performance_summary.csv", index=False)
        # Plot combined R² for all models and sheets
    plot_combined_r2_rmse(all_results)
    
    print("Summary saved to 'model_performance_summary.csv'")
    
    # Create comparison chart between Random Forest and Gradient Boosting
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    
    # FIFA Rating models comparison across sheets
    if sheets:
        sheets_idx = np.arange(len(sheets))
        diff_fifa = [gb - rf for gb, rf in zip(gb_fifa_r2_values, rf_fifa_r2_values)]
        
        bars = axes[0].bar(sheets_idx, diff_fifa, color=['g' if d > 0 else 'r' for d in diff_fifa])
        axes[0].set_title("FIFA Rating - Gradient Boosting vs Random Forest")
        axes[0].set_ylabel("GB R² - RF R² (positive = GB better)")
        axes[0].set_xticks(sheets_idx)
        axes[0].set_xticklabels(sheets, rotation=45)
        axes[0].axhline(y=0, color='k', linestyle='-', alpha=0.3)
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            axes[0].annotate(f"{height:.3f}",
                         xy=(bar.get_x() + bar.get_width() / 2, height),
                         xytext=(0, 3 if height > 0 else -10),  # 3 points vertical offset
                         textcoords="offset points",
                         ha='center', va='bottom' if height > 0 else 'top')
    
    # Real Rating models comparison across sheets
    if sheets:
        sheets_idx = np.arange(len(sheets))
        diff_real = [gb - rf for gb, rf in zip(gb_real_r2_values, rf_real_r2_values)]
        
        bars = axes[1].bar(sheets_idx, diff_real, color=['g' if d > 0 else 'r' for d in diff_real])
        axes[1].set_title("Real-Life Rating - Gradient Boosting vs Random Forest")
        axes[1].set_ylabel("GB R² - RF R² (positive = GB better)")
        axes[1].set_xticks(sheets_idx)
        axes[1].set_xticklabels(sheets, rotation=45)
        axes[1].axhline(y=0, color='k', linestyle='-', alpha=0.3)
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            axes[1].annotate(f"{height:.3f}",
                         xy=(bar.get_x() + bar.get_width() / 2, height),
                         xytext=(0, 3 if height > 0 else -10),  # 3 points vertical offset
                         textcoords="offset points",
                         ha='center', va='bottom' if height > 0 else 'top')
    
    plt.tight_layout()
    plt.savefig("Model_Comparison_Difference.png")
    plt.close()

def process_all_sheets(file_path, feature_cols, target_fifa, target_real, use_weighted_features=True):
    """
    Process all sheets in an Excel file with the RF model
    
    Args:
        file_path: Path to the Excel file
        feature_cols: List of feature columns to use
        target_fifa: Column name for FIFA rating target
        target_real: Column name for real-life rating target
        use_weighted_features: Whether to apply position-based weights
    """
    print(f"Processing file: {file_path}")
    
    # Load all sheets
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    print(f"Found {len(all_sheets)} sheets: {list(all_sheets.keys())}")
    
    # Process each sheet
    all_results = []
    
    for sheet_name, df in all_sheets.items():
        if df.empty:
            print(f"Sheet '{sheet_name}' is empty. Skipping.")
            continue
            
        result = train_and_evaluate_sheet(df, sheet_name, feature_cols, target_fifa, target_real, use_weighted_features)
        if result:
            all_results.append(result)
    
    # Compare results across sheets
    compare_sheets_results(all_results)
    
    return all_results


In [10]:
FEATURE_COLS = ['finishing', 'creativity', 'distribution', 'defense', 'duels']
TARGET_FIFA = 'Fifa Ability Overall'  
TARGET_REAL = 'Rating'

# File to process - change to your normalized or weighted file
FILE_PATH = '../notebooks/Composite_Features_Output_normalized.xlsx'  # or 'General_Weighted_Features.xlsx'

# Whether to apply position-specific weights
USE_WEIGHTED_FEATURES = True

# Process all sheets
results = process_all_sheets(FILE_PATH, FEATURE_COLS, TARGET_FIFA, TARGET_REAL, USE_WEIGHTED_FEATURES)

print("\nProcessing complete!")
print(f"Model performance summary and feature importance plots have been saved.")

# Optional: Save position-weighted data to a new Excel file
if USE_WEIGHTED_FEATURES:
    print("\nCreating position-weighted Excel file...")
    all_sheets = pd.read_excel(FILE_PATH, sheet_name=None)
    
    with pd.ExcelWriter('Position_Weighted_Features.xlsx') as writer:
        for sheet_name, df in all_sheets.items():
            if df.empty:
                continue
            
            weighted_df, _ = apply_position_weights(df, sheet_name, FEATURE_COLS)
            weighted_df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print("Position-weighted data saved to 'Position_Weighted_Features.xlsx'")


Processing file: ../notebooks/Composite_Features_Output_normalized.xlsx
Found 4 sheets: ['Data', 'DEF', 'MID', 'OFF']

===== Processing Sheet: Data =====
Using equal weights for all features.
Using weighted features: ['finishing', 'creativity', 'distribution', 'defense', 'duels']
--- Data - FIFA Rating Prediction ---
Random Forest R²: 0.219, RMSE: 4.684
Gradient Boosting R²: 0.246, RMSE: 4.600
--- Data - Real-Life Rating Prediction ---
Random Forest R²: 0.652, RMSE: 0.169
Gradient Boosting R²: 0.682, RMSE: 0.161

===== Processing Sheet: DEF =====
Applying DEF weights to sheet 'DEF'
  - Applied weight 1 to 'finishing'
  - Applied weight 1 to 'creativity'
  - Applied weight 2 to 'distribution'
  - Applied weight 4 to 'defense'
  - Applied weight 3 to 'duels'
  - Created 'weighted_score' column
Using weighted features: ['finishing_weighted', 'creativity_weighted', 'distribution_weighted', 'defense_weighted', 'duels_weighted']
--- DEF - FIFA Rating Prediction ---
Random Forest R²: 0.126, R